# Transfer Learning

Training on one task, and re-training last layers of the same model for another task. Rapid results.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

In [25]:
print(torch.cuda.is_available())
device = torch.device("cuda")

True


In [16]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [17]:
data_dir = 'data/hymenoptera_data'
image_datasets = {split: datasets.ImageFolder(os.path.join(data_dir, split), 
                        data_transforms[split]) for split in ['train', 'val']}

In [19]:
dataloaders = {split: torch.utils.data.DataLoader(image_datasets[split],
                        batch_size=4, shuffle=True, num_workers=4)
                        for split in ['train', 'val']}

In [22]:
dataset_sizes = {split: len(image_datasets[split]) for split in ['train', 'val']}
class_names = image_datasets['train'].classes

In [34]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    
    best_weights = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)
        
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            print(f'{phase} loss: {epoch_loss:.4f}, acc: {epoch_acc:.4f}')
            
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_weights = copy.deepcopy(model.state_dict())
        print()
                
    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val acc: {best_acc:.4f}')

    model.load_state_dict(best_weights)
    return model

In [27]:
model = models.resnet18(pretrained=True)

In [28]:
for param in model.parameters():
    param.requires_grad = False

In [ ]:
# Define a new final layer for the model
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 2)
model.to(device)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Scheduler to iteratively update learning rate
step_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
model = train_model(model, criterion, optimizer, step_scheduler, num_epochs=20)

Epoch 0/19
----------
train loss: 0.4050, acc: 0.7951
val loss: 0.2691, acc: 0.9216

Epoch 1/19
----------
train loss: 0.4187, acc: 0.7992
val loss: 0.2562, acc: 0.9281

Epoch 2/19
----------
train loss: 0.4243, acc: 0.7951
val loss: 0.2521, acc: 0.9281

Epoch 3/19
----------
train loss: 0.3877, acc: 0.8566
val loss: 0.2531, acc: 0.9216

Epoch 4/19
----------
train loss: 0.4152, acc: 0.8115
val loss: 0.2594, acc: 0.9281

Epoch 5/19
----------
train loss: 0.4115, acc: 0.8197
val loss: 0.2436, acc: 0.9216

Epoch 6/19
----------
train loss: 0.4086, acc: 0.8238
val loss: 0.2482, acc: 0.9346

Epoch 7/19
----------
train loss: 0.4040, acc: 0.8156
val loss: 0.2404, acc: 0.9281

Epoch 8/19
----------
train loss: 0.4315, acc: 0.7992
val loss: 0.2427, acc: 0.9346

Epoch 9/19
----------
train loss: 0.4206, acc: 0.8156
val loss: 0.2394, acc: 0.9281

Epoch 10/19
----------
train loss: 0.3692, acc: 0.8402
val loss: 0.2563, acc: 0.9281

Epoch 11/19
----------
train loss: 0.3750, acc: 0.8525
val loss: